In [69]:
import pandas as pd 

In [70]:
def read_csv(file_path: str, sep: str = ",") -> pd.DataFrame:
    """
    Đọc dữ liệu từ file CSV và trả về DataFrame.

    Parameters:
        file_path (str): Đường dẫn đến file CSV cần đọc.
        sep (str): Ký tự phân tách giữa các cột, mặc định là ','.Parameters

    Returns:
        pd.DataFrame: Dữ liệu được đọc vào từ file CSV.
    """
    try:
        df = pd.read_csv(file_path, sep=sep, encoding="utf-8")
        return df
    except FileNotFoundError:
        print(f" Không tìm thấy file tại: {file_path}")
    except Exception as e:
        print(f"Lỗi khi đọc file CSV: {e}")
    return pd.DataFrame()


def to_csv(df: pd.DataFrame, file_path: str) -> bool:
    """
    Lưu DataFrame thành file CSV sau khi làm sạch dữ liệu.

    Parameters:
        df (pd.DataFrame): DataFrame cần lưu.
        file_path (str): Đường dẫn lưu file CSV.

    Returns:
        bool: True nếu lưu thành công, False nếu gặp lỗi.
    """
    try:
        df.to_csv(file_path, index=False, encoding="utf-8")
        print(f"File đã được lưu tại: {file_path}")
        return True
    except FileNotFoundError:
        print(f"Không tìm thấy đường dẫn: {file_path}")
    except Exception as e:
        print(f"Lỗi khi lưu file CSV: {e}")
    return False

In [71]:
def extract_rating_features(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Tách điểm trung bình và số lượt đánh giá từ cột 'bus_rating'.

    Ví dụ:
        "4.7 (123)" -> rating_overall=4.7, reviewer_count=123

    Parameters:
        df (pd.DataFrame): DataFrame chứa cột đánh giá.
        col_name (str): Tên cột chứa dữ liệu đánh giá, ví dụ 'bus_rating'.

    Returns:
        pd.DataFrame: Thêm 2 cột 'rating_overall' và 'reviewer_count'.
    """
    split_cols = df[col_name].str.split(' ', expand=True)
    df['rating_overall'] = pd.to_numeric(split_cols[0], errors='coerce')
    df['reviewer_count'] = split_cols[1].replace(r'[()]', '', regex=True).astype(int)
    return df

In [72]:
def extract_number_of_seats(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Trích xuất số lượng ghế từ cột 'seat_type' và tạo cột mới 'number_of_seat'.

    Ví dụ:
        "Xe giường nằm 40 chỗ"  ->  40
        "Ghế ngồi 29 chỗ"       ->  29

    Parameters:
        df (pd.DataFrame): DataFrame chứa cột 'seat_type' mô tả loại ghế.

    Returns:
        pd.DataFrame: DataFrame đã được thêm cột 'number_of_seat'.
    """

    # Trích xuất số
    df['number_of_seat'] = df[col_name].replace(r'[^0-9]', '', regex=True)
    df['number_of_seat'] = pd.to_numeric(df['number_of_seat'], errors='coerce').astype(int)

    return df

In [73]:
def normalize_fare(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    """
    Chuẩn hoá giá vé về dạng số nguyên (VND).

    Ví dụ:
        "Từ 350.000đ" -> 350000
        "200.000đ"    -> 200000

    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu giá.
        cols (list[str]): Danh sách 2 cột ['original_price', 'sale_price'].

    Returns:
        pd.DataFrame: Thêm 2 cột mới 'price_original' và 'price_discounted'.
    """
    price = df[cols].replace(r'[^0-9]', '', regex=True)
    # fillna = 0 và ép kiểu sang int
    price = price.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)   

    df['price_original'] = price.iloc[:, 0] * 1000
    df['price_discounted'] = price.iloc[:, 1] * 1000

    return df


In [74]:
def convert_to_time(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Chuyển đổi cột thời gian từ chuỗi (string) sang kiểu datetime.time.

    Ví dụ:
        "07:30"  ->  "7:30:00"
        "21:05"  ->  "21:05:00"

    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu cần xử lý.
        col_name (str): Tên cột chứa dữ liệu thời gian (ví dụ 'departure_time').

    Returns:
        pd.DataFrame: DataFrame với cột đã được chuyển sang kiểu datetime.time.
    """

    df[col_name] = pd.to_datetime(df[col_name], format='%H:%M', errors='coerce').dt.time
    return df

In [75]:
def normalize_date(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Chuẩn hóa cột ngày tháng từ chuỗi sang định dạng 'YYYY-MM-DD'.

    Ví dụ:
        "T4, 17/10/2025"  ->  "2025-10-17"

    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu cần xử lý.
        col_name (str): Tên cột chứa dữ liệu ngày tháng (ví dụ 'departure_date').

    Returns:
        pd.DataFrame: DataFrame với cột 'departure_time' đã được chuẩn hóa.
    """

    date = df[col_name].str.split(', ', expand=True)[1]
    df[col_name] = pd.to_datetime(date, format='%d/%m/%Y', errors='coerce').dt.strftime('%Y-%m-%d')
    return df


In [76]:
def normalize_duration(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Chuẩn hóa cột thời lượng di chuyển sang tổng số phút (int).

    Ví dụ:
        "2h30m" -> 150
        "1h"    -> 60
        "45m"   -> 45

    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu cần xử lý.
        col_name (str): Tên cột chứa thời lượng di chuyển (ví dụ 'duration').

    Returns:
        pd.DataFrame: DataFrame với cột mới 'duration_m' biểu thị tổng số phút di chuyển.
    """

    # Trích xuất phần giờ và phút
    hours = df[col_name].str.extract(r'(\d+)h')[0].astype('Int64').fillna(0)
    mins = df[col_name].str.extract(r'(\d+)m')[0].astype('Int64').fillna(0)

    # Tính tổng số phút
    df['duration_m'] = hours * 60 + mins
    return df


In [77]:
def handle_bus_name(df: pd.DataFrame) -> pd.DataFrame:
    """
    Chuẩn hóa tên nhà xe bằng cách loại bỏ phần trong ngoặc.

    Ví dụ:
        "Hoàng Long (Ghế ngồi)" -> "Hoàng Long"

    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu cần xử lý.

    Returns:
        pd.DataFrame: DataFrame với cột 'bus_name' đã được làm sạch.
    """

    df['bus_name'].replace(r'\s*\([^)]+\)', '', regex=True, inplace=True)
    return df

In [78]:
def normalize_location_type(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Chuẩn hóa loại địa điểm (ví dụ: 'Bến xe', 'Văn phòng', 'Other') 
    trong cột chỉ định của DataFrame bằng cách mở rộng các từ viết tắt thông dụng.

    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu cần xử lý.
        col_name (str): Tên cột chứa dữ liệu văn bản cần chuẩn hóa
                        (ví dụ: 'pick_up_point' hoặc 'drop_off_point').

    Returns:
        pd.DataFrame: DataFrame với cột đã được chuẩn hóa loại địa điểm.
    """
    new_col = f"{col_name}_type"
    df[new_col] = df[col_name].str.lower().apply(
        lambda x: "Bến xe" if "bx" in x or "bến xe" in x
        else "Văn phòng" if "vp" in x or "văn phòng" in x
        else "Other"
    )

    return df

In [79]:
def fill_missing_ratings(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    """
    Điền giá trị thiếu cho các cột đánh giá theo trung bình của từng nhà xe.
    Nếu nhà xe không có dữ liệu, dùng trung bình toàn cột để thay thế.

    Returns:
        pd.DataFrame: DataFrame với các cột đánh giá đã được chuẩn hóa.
    """

    df[cols] = (
        df.groupby('bus_name', group_keys=False)[cols]
          .apply(lambda g: g.fillna(g.mean().round(1)))
          .fillna(df[cols].mean().round(1))
    )

    return df

In [80]:
def drop_unused_columns(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    """
    Loại bỏ các cột không cần thiết khỏi DataFrame.

    Parameters:
        df (pd.DataFrame): DataFrame cần xử lý.
        columns_to_drop (list[str]): Danh sách tên cột cần xóa.

    Returns:
        pd.DataFrame: DataFrame sau khi loại bỏ các cột không cần thiết.
    """

    # Loại bỏ các cột đã được trích xuất dữ liệu hoàn thành, không dùng tới nữa
    df.drop(columns=cols, errors='ignore', inplace=True)

    # Loại bỏ cột có toàn bộ giá trị là None (nếu có)
    df.dropna(axis=1, how='all', inplace=True)

    # Lọc dữ liệu có giá vé hoặc số người đánh giá > 0
    df = df[(df['reviewer_count'] > 0) & (df['price_original'] > 0)]
    return df

In [81]:
def preprocess_transport_data(df):
    """ 
    Tiền xử lý toàn bộ dữ liệu xe khách:
    - Chuẩn hoá rating, giá, thời gian, tên nhà xe
    - Làm sạch điểm đón/trả
    - Điền giá trị thiếu cho rating theo trung bình của từng nhà xe
    """

    df = extract_rating_features(df, 'bus_rating')  # trích xuất phần đánh giá (sao)
    df = extract_number_of_seats(df, 'seat_type')   # trích xuất ra số lượng ghế
    df = normalize_fare(df, ['original_price', 'sale_price'])   # chuẩn hoá, ép kiểu dữ liệu sang int
    df = convert_to_time(df, 'departure_time')  # chuẩn hoá thành hh:mm:00
    df = normalize_date(df, 'departure_date')   # chuẩn hoá thành yyyy-MM-DD
    df = normalize_duration(df, 'duration') # chuẩn hoá thành phút
    df = handle_bus_name(df)    # Làm sạch tên nhà xe
    df = normalize_location_type(df, 'pick_up_point')
    df = normalize_location_type(df, 'drop_of_point')

    return df

---

In [82]:
df = read_csv('../data/raw/hn_th_15102025.csv')
df

,bus_name,bus_rating,seat_type,departure_time,pick_up_point,arrival_date,arrival_time,drop_of_point,duration,original_price,...,departure_date,start_point,destination,An toàn,Thông tin chính xác,Thông tin đầy đủ,Thái độ nhân viên,Tiện nghi & thoải mái,Chất lượng dịch vụ,Đúng giờ
0,36 Limousine,4.8 (502),Limousine 11 chỗ,09:00,• Văn phòng Đồng Tàu,NaN,11:30,• Văn phòng Thanh Hóa,2h30m,230.0,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.8,4.8,4.8,4.7,4.7,4.7,4.7
1,Vân Anh Limousine,4.8 (32),Limousine Giường phòng 24 chỗ,13:00,• Bến xe Nước Ngầm,NaN,16:00,• Bến xe Thọ Xuân,3h,230.0,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.9,4.9,4.9,4.9,4.9,4.9,5.0
2,Thành Trung Limousine,4 (113),Limousine 9 chỗ,07:00,• Văn phòng Hà Nội,NaN,09:00,• Văn Phòng Thanh Hóa,2h,230.0,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3
3,Vĩnh Quang,4.8 (25),Limousine 10 chỗ,11:25,• VP Linh Đàm,NaN,13:55,• VP Thanh Hóa - 143 Lê Hồng Phong,2h30m,170.0,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,5.0,4.8,4.9,4.9,4.9,4.9,4.8
4,Thơm Phụng,4.6 (18),Giường nằm 46 chỗ,15:30,• Bến xe Nước Ngầm,NaN,17:45,• Big C Thanh Hóa,2h15m,150.0,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.9,4.6,4.8,4.8,4.6,4.5,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,Tiến Tiến,4 (11),Giường nằm 44 chỗ,19:00,• Bến xe Giáp Bát - quầy vé 22,NaN,20:50,• Ngã tư Bỉm Sơn,1h50m,NaN,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.0,3.8,3.8,4.0,3.5,3.7,NaN
431,Tiến Tiến,4 (11),Giường nằm 44 chỗ,17:50,• Bến xe Giáp Bát - quầy vé 22,NaN,19:40,• Ngã tư Bỉm Sơn,1h50m,NaN,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.0,3.8,3.8,4.0,3.5,3.7,NaN
432,Văn Minh,4.9 (149),Giường nằm 38 chỗ,10:55,• VP Yên Nghĩa,NaN,14:10,• Ngọc Lặc,3h15m,NaN,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,4.9,4.8,4.8,5.0,4.9,4.8,4.8
433,Bình Thảo,0 (0),Limousine 22 phòng,22:15,• Bến xe Nước Ngầm,(16/10),01:40,• Go Thanh Hóa,3h25m,NaN,...,"T4, 15/10/2025",Hà Nội,Thanh Hóa,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
rating_cols = ['An toàn', 'Thông tin chính xác', 'Thông tin đầy đủ','Thái độ nhân viên', 'Tiện nghi & thoải mái','Chất lượng dịch vụ', 'Đúng giờ']
unused_cols = ['arrival_time', 'arrival_date', 'bus_rating', 'seat_type', 'sale_price', 'original_price', 'percent_discount', 'duration', 'pick_up_point', 'drop_of_point']

In [84]:
df = preprocess_transport_data(df)

df = fill_missing_ratings(df, rating_cols)  # fillna()
df = drop_unused_columns(df, unused_cols)   # bỏ những cột không dùng đến/đã được trích xuất

df.drop_duplicates(keep='first', inplace=True)

C:\Users\WELCOM AN\AppData\Local\Temp\ipykernel_12836\2648517420.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(keep='first', inplace=True)


In [86]:
df

,bus_name,departure_time,departure_date,start_point,destination,An toàn,Thông tin chính xác,Thông tin đầy đủ,Thái độ nhân viên,Tiện nghi & thoải mái,Chất lượng dịch vụ,Đúng giờ,rating_overall,reviewer_count,number_of_seat,price_original,price_discounted,duration_m,pick_up_point_type,drop_of_point_type
0,36 Limousine,09:00:00,2025-10-15,Hà Nội,Thanh Hóa,4.8,4.8,4.8,4.7,4.7,4.7,4.7,4.8,502,11,230000,0,150,Văn phòng,Văn phòng
1,Vân Anh Limousine,13:00:00,2025-10-15,Hà Nội,Thanh Hóa,4.9,4.9,4.9,4.9,4.9,4.9,5.0,4.8,32,24,230000,200000,180,Bến xe,Bến xe
2,Thành Trung Limousine,07:00:00,2025-10-15,Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3,4.0,113,9,230000,210000,120,Văn phòng,Văn phòng
3,Vĩnh Quang,11:25:00,2025-10-15,Hà Nội,Thanh Hóa,5.0,4.8,4.9,4.9,4.9,4.9,4.8,4.8,25,10,170000,0,150,Văn phòng,Văn phòng
4,Thơm Phụng,15:30:00,2025-10-15,Hà Nội,Thanh Hóa,4.9,4.6,4.8,4.8,4.6,4.5,5.0,4.6,18,46,150000,0,135,Bến xe,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,Thành Trung Limousine,05:15:00,2025-10-15,Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3,4.0,113,11,480000,460000,165,Other,Văn phòng
420,Thành Trung Limousine,07:15:00,2025-10-15,Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3,4.0,113,11,480000,460000,165,Other,Văn phòng
421,Thành Trung Limousine,04:15:00,2025-10-15,Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3,4.0,113,11,480000,460000,165,Other,Văn phòng
422,Thành Trung Limousine,04:15:00,2025-10-15,Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3,4.0,113,11,530000,510000,195,Other,Văn phòng


In [87]:
df1 = read_csv('../data/clean/hn_th_15_10_2025.csv')
df1.drop(columns=['day', 'discounted_percentage', 'arrival_time'])

,bus_name,departure_time,pick_up_point,drop_of_point,start_point,destination,An toàn,Thông tin chính xác,Thông tin đầy đủ,Thái độ nhân viên,Tiện nghi & thoải mái,Chất lượng dịch vụ,Đúng giờ,rating_overall,review_count,number_of_seat,price_original,price_discounted,date,duration_m
0,36 Limousine,09:00:00,Văn phòng,Văn phòng,Hà Nội,Thanh Hóa,4.8,4.8,4.8,4.7,4.7,4.7,4.7,4.8,502,11,230000,0,15-10-2025,150
1,Vân Anh Limousine,13:00:00,Bến xe,Bến xe,Hà Nội,Thanh Hóa,4.9,4.9,4.9,4.9,4.9,4.9,5.0,4.8,32,24,230000,200000,15-10-2025,180
2,Thành Trung Limousine,07:00:00,Văn phòng,Văn phòng,Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3,4.0,113,9,230000,210000,15-10-2025,120
3,Vĩnh Quang,11:25:00,Văn phòng,Văn phòng,Hà Nội,Thanh Hóa,5.0,4.8,4.9,4.9,4.9,4.9,4.8,4.8,25,10,170000,0,15-10-2025,150
4,Thơm Phụng,15:30:00,Bến xe,Other,Hà Nội,Thanh Hóa,4.9,4.6,4.8,4.8,4.6,4.5,5.0,4.6,18,46,150000,0,15-10-2025,135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,Thành Trung Limousine,05:15:00,Other,Văn phòng,Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3,4.0,113,11,480000,460000,15-10-2025,165
370,Thành Trung Limousine,07:15:00,Other,Văn phòng,Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3,4.0,113,11,480000,460000,15-10-2025,165
371,Thành Trung Limousine,04:15:00,Other,Văn phòng,Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3,4.0,113,11,480000,460000,15-10-2025,165
372,Thành Trung Limousine,04:15:00,Other,Văn phòng,Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3,4.0,113,11,530000,510000,15-10-2025,195


In [ ]:
rename_cols = {
# ratings
    'An toàn': 'rating_safety',
    'Thông tin chính xác': 'rating_info_accuracy',
    'Thông tin đầy đủ': 'rating_info_completeness',
    'Thái độ nhân viên': 'rating_staff_attitude',
    'Tiện nghi & thoải mái': 'rating_comfort',
    'Chất lượng dịch vụ': 'rating_service_quanlity',
    'Đúng giờ': 'rating_punctuality',
# trips info
    'bus_name': 'company_name',
    'pick_up_point_type': 'pickup_point',
    'drop_of_point_type': 'dropoff_point'
}


In [89]:
df.rename(columns=rename_cols, inplace=True)

C:\Users\WELCOM AN\AppData\Local\Temp\ipykernel_12836\1635787294.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns=rename_cols, inplace=True)


In [90]:
df

,company_name,departure_time,departure_date,start_point,destination,rating_safety,rating_info_accuracy,rating_info_completeness,rating_staff_attitude,rating_comfort,rating_service_quanlity,rating_punctuality,rating_overall,reviewer_count,number_of_seat,price_original,price_discounted,duration_m,pickup_point,dropoff_point
0,36 Limousine,09:00:00,2025-10-15,Hà Nội,Thanh Hóa,4.8,4.8,4.8,4.7,4.7,4.7,4.7,4.8,502,11,230000,0,150,Văn phòng,Văn phòng
1,Vân Anh Limousine,13:00:00,2025-10-15,Hà Nội,Thanh Hóa,4.9,4.9,4.9,4.9,4.9,4.9,5.0,4.8,32,24,230000,200000,180,Bến xe,Bến xe
2,Thành Trung Limousine,07:00:00,2025-10-15,Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3,4.0,113,9,230000,210000,120,Văn phòng,Văn phòng
3,Vĩnh Quang,11:25:00,2025-10-15,Hà Nội,Thanh Hóa,5.0,4.8,4.9,4.9,4.9,4.9,4.8,4.8,25,10,170000,0,150,Văn phòng,Văn phòng
4,Thơm Phụng,15:30:00,2025-10-15,Hà Nội,Thanh Hóa,4.9,4.6,4.8,4.8,4.6,4.5,5.0,4.6,18,46,150000,0,135,Bến xe,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,Thành Trung Limousine,05:15:00,2025-10-15,Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3,4.0,113,11,480000,460000,165,Other,Văn phòng
420,Thành Trung Limousine,07:15:00,2025-10-15,Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3,4.0,113,11,480000,460000,165,Other,Văn phòng
421,Thành Trung Limousine,04:15:00,2025-10-15,Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3,4.0,113,11,480000,460000,165,Other,Văn phòng
422,Thành Trung Limousine,04:15:00,2025-10-15,Hà Nội,Thanh Hóa,4.4,3.9,4.0,4.3,4.3,3.9,4.3,4.0,113,11,530000,510000,195,Other,Văn phòng
